### Input Redshift Cluster Endpoint and User
Please input your redshift cluster endpoint and existing user on that cluster.

In [ ]:
REDSHIFT_ENDPOINT = 'redshift-cluster.xxxxxxxxxx.us-east-1.redshift.amazonaws.com:5439/dev'
REDSHIFT_USER="awsuser"

### Setup Run SQL function using Redshift Data API to get SQL query output directly into pandas dataframe
In this step, we are creating function run_sql, which we will use to get SQL query output directly into pandas dataframe. We will also use this function to run DDL statements

In [ ]:

import boto3
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

session = boto3.session.Session()
region = session.region_name


def run_sql(sql_text):
    client = boto3.client("redshift-data")
    res = client.execute_statement(Database=REDSHIFT_ENDPOINT.split('/')[1], DbUser=REDSHIFT_USER, Sql=sql_text,
                                   ClusterIdentifier=REDSHIFT_ENDPOINT.split('.')[0])
    query_id = res["Id"]
    done = False
    while not done:
        time.sleep(1)
        status_description = client.describe_statement(Id=query_id)
        status = status_description["Status"]
        if status == "FAILED":
            raise Exception('SQL query failed:' + query_id + ": " + status_description["Error"])
        elif status == "FINISHED":
            if status_description['ResultRows']>0:
                results = client.get_statement_result(Id=query_id)
                column_labels = []
                for i in range(len(results["ColumnMetadata"])): column_labels.append(results["ColumnMetadata"][i]['label'])
                records = []
                for record in results.get('Records'):
                    records.append([list(rec.values())[0] for rec in record])
                df = pd.DataFrame(np.array(records), columns=column_labels)
                return df
            else:
                return query_id


## Data Preparation Script
Data preparation script to be run on Redshift

In [ ]:
setup_script = """

--train table 
CREATE TABLE banknoteauthentication_train(
variance FLOAT,
skewness FLOAT,
curtosis FLOAT,
entrophy FLOAT,
class INT);

--Load 
COPY banknoteauthentication_train FROM 's3://redshiftbucket-ml-sagemaker/banknote_authentication/train_data/' IAM_ROLE '<your-amazon-redshift-sagemaker-iam-role-arn>' REGION 'us-west-2' IGNOREHEADER 1 CSV;

--test table 
CREATE TABLE banknoteauthentication_test(
variance FLOAT,
skewness FLOAT,
curtosis FLOAT,
entrophy FLOAT,
class INT);

--Load 
COPY banknoteauthentication_test FROM 's3://redshiftbucket-ml-sagemaker/banknote_authentication/test_data/' IAM_ROLE '<your-amazon-redshift-sagemaker-iam-role-arn>' REGION 'us-west-2' IGNOREHEADER 1 CSV;
 
"""

### Run data preparation script in Redshift

In [ ]:
sql_stmt = setup_script.split(";")
for sql_text in sql_stmt[:-1]:
    run_sql(sql_text);

### Run Create Model statement to create a new ML model with Redshift ML

In [ ]:
run_sql("""

--Create model 
CREATE MODEL model_banknoteauthentication_xgboost_binary FROM banknoteauthentication_train 
TARGET class 
FUNCTION func_model_banknoteauthentication_xgboost_binary 
IAM_ROLE '<<your-amazon-redshift-sagemaker-iam-role-arn>>' 
AUTO OFF 
MODEL_TYPE xgboost 
OBJECTIVE 'binary:logistic' 
PREPROCESSORS 'none' 
HYPERPARAMETERS DEFAULT EXCEPT(NUM_ROUND '100') 
SETTINGS(S3_BUCKET '<<your-amazon-s3-bucket-name>>');
""");

### View Model Progress 

In [ ]:
df = run_sql("SHOW MODEL model_banknoteauthentication_xgboost_binary")
df

## Compare accuracy of the model 


In [ ]:
df = run_sql("""
-- check accuracy 
WITH infer_data AS (
SELECT class AS label,
func_model_banknoteauthentication_xgboost_binary (variance, skewness, curtosis, entrophy) AS predicted,
CASE 
   WHEN label IS NULL
       THEN 0
   ELSE label
   END AS actual,
CASE 
   WHEN actual = predicted
       THEN 1::INT
   ELSE 0::INT
   END AS correct
FROM banknoteauthentication_test),
aggr_data AS (
SELECT SUM(correct) AS num_correct,
COUNT(*) AS total
FROM infer_data) 
SELECT (num_correct::FLOAT / total::FLOAT) AS accuracy FROM aggr_data;
""");
df


## Predict the count of original vs. counterfeit banknotes


In [ ]:
df = run_sql("""
--check prediction 
WITH infer_data AS (
    SELECT func_model_banknoteauthentication_xgboost_binary(variance, skewness, curtosis, entrophy) AS predicted
    FROM banknoteauthentication_test
    )SELECT CASE 
        WHEN predicted = '0'
            THEN 'Original banknote'
        WHEN predicted = '1'
            THEN 'Counterfeit banknote'
        ELSE 'NA'
        END AS banknote_authentication
    ,COUNT(1) AS count FROM infer_data GROUP BY 1;
""");
df
